In [349]:
import os
import random
import time
import pandas as pd
import numpy as np
import mxnet as mx
from mxnet import autograd, gluon, image, init, nd
from mxnet.gluon import data as gdata, loss as gloss, nn, utils as gutils

In [395]:
labelnames = 'ret_1|ret_1_alpha|ret_1_label|ret_1_weight|ret_2|ret_2_alpha|ret_2_label|ret_2_weight|ret_3|ret_3_alpha|ret_3_label|ret_3_weight|ret_4|ret_4_alpha|ret_4_label|ret_4_weight|ret_5|ret_5_alpha|ret_5_label|ret_5_weight'
label = 'ret_1_label'
weight = 'ret_1_weight'
labelnames = labelnames.split("|")
start_train = '20181227'
end_train = '20181231'
start_test = '20190104'
end_test = '20190107'
datadir = '~\\Documents\\NewData\\data4nn_pv\\data4nn_pv_{YYYYMMDD}.h5'
batch_size = 512
lr = 0.05
num_epochs = 5
ctx = mx.cpu()

In [363]:
def data_iter(batch_size, features, labels):
    """Iterate through a data set."""
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield features.take(j, axis=0), labels.take(j, axis=0)

In [364]:
def load_data(startDate, endDate, labelnames, label, weight, datadir):
    """read the data and then load into memory."""
    path = os.path.expanduser(datadir)
    date_range = pd.date_range(startDate, endDate, closed='left')
    dfs = []
    for date in date_range:
        file = path.format(YYYYMMDD=date.strftime("%Y%m%d"))
        if os.path.exists(file):
            df = pd.read_hdf(file)
            dfs.append(df)
    rawdata = pd.concat(dfs)
    labels = rawdata[label]
    weight = rawdata[weight]
    features = rawdata.drop(labels=labelnames, axis=1)
    labels = nd.array(labels)
    weight = nd.array(weight)
    features = nd.array(features)
    return features, labels, weight

In [396]:
features_train, labels_train, weight_train = load_data(startDate=start_train, endDate=end_train, labelnames=labelnames, label=label, weight=weight, datadir=datadir)
features_test, labels_test, weight_test = load_data(startDate=start_test, endDate=end_test, labelnames=labelnames, label=label, weight=weight, datadir=datadir)

In [366]:
features_train = features_train.reshape((-1, 6, 60))
features_test = features_test.reshape((-1, 6, 60))
#features = features.swapaxes(2, 3)

In [370]:
class CovNet(nn.Block): 
    def __init__(self, channels, kernel_size, strides=1, **kwargs):
        super(CovNet, self).__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(nn.BatchNorm(),
                     nn.Conv1D(channels, kernel_size, padding=1, strides=strides),
                     nn.BatchNorm(), 
                     nn.Activation('relu'),
                     nn.MaxPool1D(pool_size=3, strides=1, padding=1))
    def forward(self, X):
        Y = self.net(X)
        return Y

In [371]:
class WideNet(nn.Block):
    # c1 - c4为每条线路里全连接层节点个数
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(WideNet, self).__init__(**kwargs)
        
        # 线路1，512 x 1全连接层
        self.p1 = nn.Sequential()
        self.p1.add(nn.Dense(c1[0], activation='relu'),
                    nn.Dense(c1[1], activation='relu'))
        
        # 线路2，256 x 2全连接层
        self.p2 = nn.Sequential()
        self.p2.add(nn.Dense(c2[0], activation='relu'),
                    nn.Dense(c2[1], activation='relu'))
        
        # 线路3，128 x 3全连接层
        self.p3 = nn.Sequential()
        self.p3.add(nn.Dense(c3[0], activation='relu'),
                    nn.Dense(c3[1], activation='relu'))

        # 线路4，64 x 4全连接层
        self.p4 = nn.Sequential()
        self.p4.add(nn.Dense(c4[0], activation='relu'),
                    nn.Dense(c4[1], activation='relu'))
        
    def forward(self, x):
        p1 = self.p1(x)
        p2 = self.p2(x)
        p3 = self.p3(x)
        p4 = self.p4(x)
        p = nd.concat(p1, p2, p3, p4, dim=1) 
#         print(self.p1.name, 'output shape:\t', p1.shape)
#         print(self.p2.name, 'output shape:\t', p2.shape)
#         print(self.p3.name, 'output shape:\t', p3.shape)
#         print(self.p4.name, 'output shape:\t', p4.shape)
#         print('p output shape:\t', p.shape)
        return p

In [386]:
net = nn.Sequential()
net.add(CovNet(16,3),
        CovNet(32,3),
        nn.Flatten(),
        WideNet(c1=[512,1], c2=[256,2], c3=[128,3], c4=[64,4]),
        nn.Dense(2, activation='sigmoid'),
       )
net.initialize(force_reinit=True, init=init.Xavier())
print(net.collect_params())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

sequential116_ (
  Parameter batchnorm66_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm66_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm66_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm66_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter conv40_weight (shape=(16, 0, 3), dtype=<class 'numpy.float32'>)
  Parameter conv40_bias (shape=(16,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm67_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm67_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm67_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm67_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm68_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm68_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter batchnorm68_running_mean (shape=(0,), dtype=<class 'numpy.float

In [373]:
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n

In [374]:
def train_ms(net, train_iter, test_iter, batch_size, trainer, num_epochs, ctx=mx.cpu()):
    """Train and evaluate a model with CPU or GPU."""
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [375]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

In [376]:
train_iter = data_iter(batch_size=batch_size, features=features_train, labels=labels_train)
test_iter = data_iter(batch_size=batch_size, features=features_test, labels=labels_test)

In [397]:
train_ms(net, train_iter, test_iter, batch_size, trainer, num_epochs, ctx=ctx)

KeyboardInterrupt: 

In [292]:
X = feature
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

Conv1D(6 -> 16, kernel_size=(3,), stride=(1,), padding=(1,))
covnet23 output shape:	 (3, 16, 60)
Conv1D(16 -> 32, kernel_size=(3,), stride=(1,), padding=(1,))
covnet24 output shape:	 (3, 32, 60)


In [124]:
features_back[['open_1','high_1', 'low_1', 'avg_1', 'close_1', 'volume_1' ]]

open_1  high_1  low_1    avg_1  close_1    volume_1
dates      instruments                                                     
2018-11-05 000001.SZ     11.04   11.16  10.83  11.0224    11.09  2212922.87
           000002.SZ     25.20   25.25  24.00  24.5443    24.62   652339.22
           000004.SZ     15.50   15.94  15.39  15.5786    15.76    12594.87
           000005.SZ      2.73    2.76   2.73   2.7462     2.76    47221.45
           000006.SZ      5.10    5.15   5.06   5.1127     5.14   143410.08
           000007.SZ      7.39    7.50   7.13   7.2262     7.17   234180.19
           000008.SZ      4.37    4.43   4.31   4.3564     4.37   405368.64
           000009.SZ      4.10    4.15   4.09   4.1274     4.14   148234.88
           000010.SZ      4.36    4.39   4.30   4.3374     4.38    39959.00
           000011.SZ      9.00    9.06   8.90   8.9913     9.06    46312.51
           000012.SZ      4.21    4.28   4.21   4.2480     4.28   161668.31
           000014.SZ      9.08    9.23   9.06   9.1370     9.22    67649.78
           000016.SZ      3.56    3.62   3.55   3.5920     3.61   121101.69
           000017.SZ      4.39    4.49   4.36   4.4149     4.41   156477.25
           000018.SZ      2.03    2.11   2.03   2.0635     2.07   257631.15
           000019.SZ      7.54    7.68   7.52   7.5954     7.65    82179.35
           000020.SZ     10.29   10.49  10.24  10.3816    10.46    18066.08
           000021.SZ      5.50    5.80   5.50   5.6354     5.73   140583.11
           000023.SZ     10.95   11.24  10.80  11.0428    11.13    33248.90
           000025.SZ     26.37   26.80  26.15  26.4910    26.69    46506.35
           000026.SZ      7.12    7.25   7.09   7.1614     7.25    30433.10
           000027.SZ      5.09    5.17   5.08   5.1433     5.17    97749.26
           000028.SZ     46.45   47.58  46.45  47.2519    47.27    19045.84
           000029.SZ     11.17   11.17  11.17  11.1700    11.17        0.00
           000030.SZ      3.80    3.92   3.79   3.8516     3.87    57100.24
           000031.SZ      5.01    5.05   4.93   4.9992     5.05   178062.92
           000032.SZ      6.79    6.93   6.70   6.8452     6.92    22215.83
           000034.SZ     14.00   14.97  14.00  14.3254    14.76    62552.38
           000035.SZ      4.65    4.72   4.57   4.6149     4.64    75571.06
           000036.SZ      4.92    4.99   4.86   4.9329     4.95    90835.78
...                        ...     ...    ...      ...      ...         ...
2018-12-28 603936.SH     14.60   15.19  14.40  14.8086    14.41    22544.30
           603937.SH     11.72   11.76  11.18  11.5507    11.18    25901.10
           603938.SH     19.31   19.46  18.88  19.1863    18.97    22505.00
           603939.SH     42.60   43.38  42.21  42.6961    42.47    16932.46
           603955.SH     14.75   14.89  14.13  14.5056    14.15     5727.10
           603958.SH      8.21    8.32   7.96   8.2405     7.97    19606.00
           603959.SH     15.75   15.75  14.65  15.0799    14.77    17247.03
           603960.SH     26.79   27.78  26.13  27.1598    26.22     8522.47
           603963.SH     12.39   12.43  11.68  12.1528    11.79     7005.00
           603966.SH      9.15    9.38   8.69   9.0111     8.71    21704.48
           603968.SH     12.95   13.02  12.52  12.8029    12.57     5613.00
           603969.SH      4.87    4.91   4.68   4.7673     4.70    56722.85
           603970.SH     15.01   15.34  14.96  15.1690    15.10    29291.32
           603976.SH     16.57   16.78  16.31  16.4310    16.31    22221.55
           603977.SH      7.19    7.19   6.90   7.0383     6.90     6128.80
           603978.SH     19.99   20.10  19.06  19.5988    19.06    10152.00
           603979.SH      7.69    7.72   7.45   7.5585     7.50    10995.65
           603980.SH     12.01   12.05  11.77  11.9032    11.77    11034.00
           603985.SH     21.28   21.49  21.00  21.3287    21.11    15711.42
           603986.SH     66.41   66.60  62.00  64.2743    62.30    57670

In [113]:
features_back

open_0  open_1  open_2  open_3  open_4  open_5  \
dates      instruments                                                   
2018-11-05 000001.SZ     10.95   11.04   10.99   10.95   10.78   11.20   
           000002.SZ     24.48   25.20   24.90   23.68   23.31   24.28   
           000004.SZ     16.00   15.50   15.63   15.50   15.49   17.49   
           000005.SZ      2.76    2.73    2.74    2.69    2.66    2.72   
           000006.SZ      5.12    5.10    5.13    5.00    4.92    5.08   
           000007.SZ      7.20    7.39    7.25    7.13    6.99    7.39   
           000008.SZ      4.41    4.37    4.43    4.29    4.25    4.27   
           000009.SZ      4.15    4.10    4.05    4.00    3.92    4.04   
           000010.SZ      4.37    4.36    4.38    4.38    4.34    4.40   
           000011.SZ      9.02    9.00    8.90    8.78    8.72    8.95   
           000012.SZ      4.31    4.21    4.21    4.13    4.11    4.14   
           000014.SZ      9.22    9.08    9.03    8.98    8.96    9.20   
           000016.SZ      3.59    3.56    3.59    3.62    3.44    3.47   
           000017.SZ      4.44    4.39    4.32    4.14    4.11    4.28   
           000018.SZ      2.10    2.03    2.04    2.02    2.02    2.03   
           000019.SZ      7.65    7.54    7.49    7.39    7.40    7.50   
           000020.SZ     10.49   10.29   10.16    9.96   10.08   10.14   
           000021.SZ      5.73    5.50    5.44    5.27    5.13    5.16   
           000023.SZ     11.18   10.95   10.85   10.73   10.60   10.60   
           000025.SZ     26.60   26.37   26.20   25.68   25.33   25.80   
           000026.SZ      7.21    7.12    7.05    6.97    6.87    7.02   
           000027.SZ      5.17    5.09    5.20    5.18    5.16    5.20   
           000028.SZ     47.02   46.45   46.05   46.00   44.99   43.44   
           000029.SZ     11.17   11.17   11.17   11.17   11.17   11.17   
           000030.SZ      3.84    3.80    3.73    3.69    3.60    3.82   
           000031.SZ      5.01    5.01    5.01    4.91    4.88    5.17   
           000032.SZ      6.87    6.79    6.76    6.58    6.49    6.68   
           000034.SZ     14.80   14.00   13.65   13.45   13.31   13.35   
           000035.SZ      4.68    4.65    4.53    4.53    4.53    4.66   
           000036.SZ      4.92    4.92    4.70    4.58    4.51    4.50   
...                        ...     ...     ...     ...     ...     ...   
2018-12-28 603936.SH     14.31   14.60   14.75   14.17   14.23   13.96   
           603937.SH     11.21   11.72   11.43   11.53   11.52   11.48   
           603938.SH     18.89   19.31   19.28   18.56   18.25   18.55   
           603939.SH     42.45   42.60   43.97   43.80   41.90   43.40   
           603955.SH     14.23   14.75   14.60   14.99   15.00   15.55   
           603958.SH      7.94    8.21    8.10    8.13    8.02    7.94   
           603959.SH     14.80   15.75   15.69   15.81   15.97   15.97   
           603960.SH     26.39   26.79   27.04   27.23   26.90   27.49   
           603963.SH     11.70   12.39   12.40   12.70   12.38   12.20   
           603966.SH      8.66    9.15    8.94    9.11    9.40    9.33   
           603968.SH     12.58   12.95   12.85   13.09   13.06   13.07   
           603969.SH      4.70    4.87    4.77    4.74    4.68    4.67   
           603970.SH     15.20   15.01   14.68   14.77   14.90   14.46   
           603976.SH     16.32   16.57   17.05   17.30   17.26   17.21   
           603977.SH      6.90    7.19    7.05    7.08    7.17    7.30   
           603978.SH     19.13   19.99   20.00   20.15   20.10   19.86   
           603979.SH      7.47    7.69    7.58    7.50    7.57    7.65   
           603980.SH     11.79   12.01   11.84   11.90   12.02   11.99   
           603985.SH     21.10   21.28   20.73   20.85   20.22   20.24   
           603986.SH     62.30   66.41   66.80   66.64   66.50   67.95   
           603987.SH      6.44    6.52    6.50    6.63    6.78    6.60   
           603988.SH      9.

In [88]:
features.columns

Index(['open_0', 'open_1', 'open_2', 'open_3', 'open_4', 'open_5', 'open_6',
       'open_7', 'open_8', 'open_9',
       ...
       'volume_50', 'volume_51', 'volume_52', 'volume_53', 'volume_54',
       'volume_55', 'volume_56', 'volume_57', 'volume_58', 'volume_59'],
      dtype='object', length=360)